In [1]:
############################################ Import Python Libraries Required ############################################ 
import numpy as np
import pandas as pd
import scipy

import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import matplotlib.colors

import datetime
import os 
from os import path
import seaborn as sns
from scipy import signal

import warnings
warnings.filterwarnings('ignore')

import pickle

import statistics
import math 
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)
# %config InlineBackend.figure_format = 'retina'
from PIL import Image



In [28]:
############################# Update Actual Cases #############################################3
import requests
os.system("rm state_wise_daily.csv")
url = """https://data.covid19india.org/csv/latest/state_wise_daily.csv"""
response = requests.get(url)


with open( "state_wise_daily.csv", 'wb') as f:
    f.write(response.content)
df  = pd.read_csv("state_wise_daily.csv",delimiter=",")

listOfColumns  = ["Date","Date_YMD","Status","KA"]
df = df[listOfColumns].copy(deep=True)
df["Date"] = pd.to_datetime(df["Date"])

## Sum all the values upto Jul-1st 2020 and add that as cumuliative total to Jul-1st 2020
## The data fitted to the model starts only from Jul-1st 2020

## Sum of COnfirmed 
df_temp = df[(df["Date"] <= '2020-07-01') & (df["Status"]=="Confirmed")]
ConfirmVal = df_temp["KA"].cumsum().iloc[-1]
index = df.loc[(df["Date"] =='2020-07-01') & (df["Status"] == "Confirmed")]["KA"].index
df.loc[index,"KA"] = ConfirmVal

## Sum of COnfirmed  
df_temp = df[(df["Date"] <= '2020-07-01') & (df["Status"]=="Recovered")]
RecoverVal = df_temp["KA"].cumsum().iloc[-1]
index = df.loc[(df["Date"] =='2020-07-01') & (df["Status"] == "Recovered")]["KA"].index
df.loc[index,"KA"] = RecoverVal

## Sum of Deceased 
df_temp = df[(df["Date"] <= '2020-07-01') & (df["Status"]=="Deceased")]
DeceaseVal = df_temp["KA"].cumsum().iloc[-1]
index = df.loc[(df["Date"] =='2020-07-01') & (df["Status"] == "Deceased")]["KA"].index
## -2 , HARDCODED for adjustment with the data used to fir the model ( The current data we have has 253 as Total Deceased as)
df.loc[index,"KA"] = DeceaseVal - 2   


#Copy data only from Jun-01-2020
df = df[(df["Date"] >= '2020-07-01')].copy(deep=True)

##reset index of Dataframe
df.reset_index(inplace=True,drop=True)


## get the KA columns from the dataframe
totaldata_all = df.KA
totalDays  = int(len(totaldata_all)/3)

N_dataDays = totalDays

##Declare Arrays 
actualAct = np.zeros(N_dataDays) 
actualRecov = np.zeros(N_dataDays)  
actualTot = np.zeros(N_dataDays) 
actualDes = np.zeros(N_dataDays) 

## Get the first day data
MarInitTot = totaldata_all[0]
MarInitRecov = totaldata_all[1]
MarInitDes = totaldata_all[2]

##initial Setup
actualTot[0] = MarInitTot 
actualRecov[0] = MarInitRecov 
actualDes[0] = MarInitDes 
actualAct[0] = actualTot[0] - actualRecov[0] - actualDes[0]


for idx in range(1, N_dataDays):
    actualTot[idx] = actualTot[idx-1] + totaldata_all[3*(idx)]
    actualRecov[idx] = actualRecov[idx-1] + totaldata_all[3*(idx)+1]
    actualDes[idx] = actualDes[idx-1] + totaldata_all[3*(idx)+2]
    actualAct[idx] = actualTot[idx] - actualRecov[idx] - actualDes[idx]


ActualDict = {}
ActualDict["Actual"] = actualAct
ActualDict["Deceased"] = actualDes
ActualDict["Recovered"] = actualRecov
ActualDict["Cumuliative"] = actualTot

file_name = "Data_Files/actual_Data.pkl"

open_file = open(file_name, "wb")
pickle.dump(ActualDict, open_file)
open_file.close()


In [29]:
# ActualDict['Actual'] = np.append(ActualDict['Actual'],[8512.0,8370.0,8306.0,8267.0,8188.0,8090.0,8002.0,7989.0,7955.0,8027.0,8017.0,8036.0,8027.0,7997.0,7912.0,7493.0,7409])
# ActualDict['Actual']

array([  8196.,   9408.,  10610.,  11968.,  13253.,  14386.,  15298.,
        16528.,  17782.,  19035.,  20883.,  22746.,  24573.,  25840.,
        27853.,  30655.,  33205.,  36631.,  39370.,  42216.,  44140.,
        47069.,  49931.,  52791.,  55388.,  58417.,  61819.,  64434.,
        67450.,  69702.,  72007.,  73221.,  74592.,  74471.,  73848.,
        73960.,  75070.,  77689.,  79768.,  80976.,  79911.,  79609.,
        80345.,  78339.,  79203.,  81278.,  81522.,  80653.,  79792.,
        81107.,  82159.,  83076.,  82687.,  83561.,  81224.,  82423.,
        83621.,  85000.,  86360.,  86459.,  88104.,  87248.,  91012.,
        94472.,  96111.,  99114.,  99630.,  99279.,  97014.,  96931.,
        99483., 101550.,  98339.,  97828.,  99216.,  98476.,  98549.,
       101639., 103644., 101142.,  98577.,  98056.,  95348.,  93166.,
        94665.,  95562.,  98487., 101795., 104737., 104061., 107750.,
       107629., 110425., 111999., 112796., 115587., 115490., 115164.,
       116166., 1171

In [30]:
# file_name = "Data_Files/actual_Data.pkl"

# open_file = open(file_name, "wb")
# pickle.dump(ActualDict, open_file)
# open_file.close()

In [32]:
pip install tabula-py


Note: you may need to restart the kernel to use updated packages.


In [53]:
import tabula as tb
import re
import requests

url = """https://covid19.karnataka.gov.in/storage/pdf-files/EMB-NOV21//16-11-2021%20HMB%20English.pdf"""
response = requests.get(url)


with open( "a.pdf", 'wb') as f:
    f.write(response.content)
file = 'a.pdf'

import pandas as pd
data = tb.read_pdf(file,area=(85,20,700,580),columns=(1,40,130,190,250,320,370,420,470,520,580),pages='5',pandas_options={'header':None})

data


[    0    1               2        3        4    5        6      7        8   \
 0  NaN  NaN             NaN      NaN      NaN  NaN      NaN    NaN      NaN   
 1  NaN  NaN             NaN      NaN      NaN  NaN      NaN    NaN  Today’s   
 2  NaN  NaN             NaN      NaN      NaN  NaN      NaN  Total      NaN   
 3  NaN  Sl.             NaN  Today’s    Total  NaN      NaN    NaN      NaN   
 4  NaN  NaN             NaN      NaN      NaN  NaN      NaN    NaN      NaN   
 ..  ..  ...             ...      ...      ...  ...      ...    ...      ...   
 40 NaN   28  Uttara Kannada        8    56346    5    55487     88        0   
 41 NaN   29      Vijayapura        0    36274    0    35774      5        0   
 42 NaN   30        Yadagiri        0    27546    0    27338      2        0   
 43 NaN   31         Others*        0       36    0       33      0        0   
 44 NaN  NaN           Total      255  2992276  667  2946601   7493        7   
 
        9       10  
 0     NaN   Deat

In [54]:
for i in range(len(data[0][1])):
    if(data[0][1][i]=='1'):
        index = i
SlNo = [int(x) for x in data[0][1][index:index+33] if (str(x)!='nan')]
district = [x for x in data[0][2][index:index+33] if (str(x)!='nan')]
district = [x for x in district if (str(x)!='Kannada')]
district[10] = 'Dakshina Kannada'
dailyPositive = [int(x) for x in data[0][3][index:index+33] if (str(x)!='nan')]

totalPositive = [int(x) for x in data[0][4][index:index+33] if (str(x)!='nan')]
dailyDischarge = [int(x) for x in data[0][5][index:index+33] if (str(x)!='nan')]
totalDischarge = [int(x) for x in data[0][6][index:index+33] if (str(x)!='nan')]
totalActive = [int(x) for x in data[0][7][index:index+33] if (str(x)!='nan')]
dailyDeaths= [int(x) for x in data[0][8][index:index+33] if (str(x)!='nan')]
totalDeaths= [int(x) for x in data[0][9][index:index+33] if (str(x)!='nan')]

In [63]:

open_file = open('districts.pkl', "wb")
pickle.dump(district, open_file)

In [65]:
with open('~/Documents/COVIDW3/COVID19_Karnataka_Data/Data/DataFrames/districts.pkl', 'rb') as f:
        ad = pickle.load(f)

ad

FileNotFoundError: [Errno 2] No such file or directory: '~/Documents/COVIDW3/COVID19_Karnataka_Data/Data/DataFrames/districts.pkl'

In [22]:
data[0][1][9:9+33]

9      1
10     2
11     3
12     4
13     5
      ..
37    27
38    28
39    29
40    30
41    31
Name: 1, Length: 33, dtype: object

In [178]:
df = pd.DataFrame()
df['SrNo'] = SlNo
df['District'] = district
df['Daily Positive'] = dailyPositive
df['Total Positive'] = totalPositive
df['Daily Discharged'] = dailyDischarge
df['Total Discharged'] = totalDischarge
df['Active'] = totalActive
df['Daily Deaths'] = dailyDeaths
df['Total Deaths'] = totalDeaths
df



,SrNo,District,Daily Positive,Total Positive,Daily Discharged,Total Discharged,Active,Daily Deaths,Total Deaths
0,1,Bagalakote,0,35180,2,34844,3,0,333
1,2,Ballari,1,97785,0,95981,118,0,1686
2,3,Belagavi,3,79946,5,78946,58,1,942
3,4,Bengaluru Rural,1,62063,2,61118,55,0,890
4,5,Bengaluru Urban,163,1254256,574,1231780,6161,2,16314
5,6,Bidar,0,24342,0,23937,0,0,401
6,7,Chamarajanagara,1,33024,3,32489,22,0,502
7,8,Chikkaballapura,3,43968,5,43527,9,0,431
8,9,Chikkamagaluru,3,51170,2,50726,48,0,396
9,10,Chitradurga,1,36693,3,36455,31,1,207


In [177]:
totalDeaths

[333,
 1686,
 942,
 890,
 16314,
 401,
 502,
 431,
 396,
 207,
 1684,
 608,
 1318,
 319,
 1256,
 643,
 820,
 330,
 635,
 524,
 654,
 2420,
 332,
 321,
 1090,
 1134,
 488,
 771,
 495,
 206,
 3]

In [130]:
df = pd.DataFrame()
df

""
